In [ ]:
# RUN THIS ONLY
!pip install -q --upgrade git+https://github.com/keras-team/keras-cv tensorflow pycocotools seaborn
!pip install tensorflow==2.15.0

In [ ]:
import tensorflow as tf
import numpy as np
import math
import keras
import keras_cv
from keras_cv import visualization
import keras
import cv2
import tensorflow
import os
import glob
import json
from collections import defaultdict
import tensorflow as tf
from keras import optimizers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
table = {}
class_mapping = {}
counter = 0

def lut(label):
    global counter
    if label in table:
        return table[label]
    counter += 1
    table[label] = counter
    class_mapping[counter] = label
    return table[label]

In [ ]:
splits = {
    'train': '/content/drive/MyDrive/Insulator Defect/Data/train',
    'test': '/content/drive/MyDrive/Insulator Defect/Data/test',
    'validation': '/content/drive/MyDrive/Insulator Defect/Data/valid'
}

def load_image(filepath):
  image_data = tf.io.read_file(filepath)
  return tf.cast(tf.io.decode_jpeg(image_data, channels=3), tf.float32)

def load(*, split, bounding_box_format):
  if not split in splits:
    raise ValueError(
        f"Invalid split provided, `split={split}`. "
        f"Expected one of {list(splits.keys())}"
    )

  path = splits[split]
  with open(f'{path}/annotations.json', 'r') as f:
    file_annotations = json.load(f)

  def generator():
    for entry in file_annotations:
      annotations = entry['annotations']
      image_path = entry['image']

      box_labels = []
      class_labels = []

      for annotation in annotations:
        box = annotation['coordinates']
        box = tf.constant(
            [box['x'], box['y'], box['width'], box['height']], tf.float32
        )
        box_labels.append(
          box
        )
        class_labels.append(
            tf.constant(lut(annotation['label']), tf.float32)
        )

      if len(box_labels) == 0:
        continue

      bounding_boxes = {
          'boxes': tf.stack(box_labels),
          'classes': tf.stack(class_labels)
      }
      image = load_image(f"{path}/{image_path}")
      bounding_boxes = keras_cv.bounding_box.convert_format(bounding_boxes, source ='xywh', target=bounding_box_format)
      yield {
          'images': image,
          'bounding_boxes': bounding_boxes
      }

  output_spec = {
    'images': tf.TensorSpec(shape=(None, None, 3)),
    'bounding_boxes': {
        'boxes': tf.TensorSpec(shape=(None, 4)),
        'classes': tf.TensorSpec(shape=(None,))
    }
  }
  return tf.data.Dataset.from_generator(generator, output_signature=output_spec)

In [ ]:
train_ds = load(split='train', bounding_box_format='xywh')
train_ds = train_ds.ragged_batch(16)
train_ds

In [ ]:
def visualize_dataset(inputs, value_range, rows, cols, bounding_box_format):
    inputs = next(iter(inputs.take(1)))
    images, bounding_boxes = inputs["images"], inputs["bounding_boxes"]
    visualization.plot_bounding_box_gallery(
        images.to_tensor(),
        value_range=value_range,
        rows=rows,
        cols=cols,
        y_true=bounding_boxes,
        scale=10,
        font_scale=6,
        bounding_box_format=bounding_box_format,
        class_mapping=class_mapping,
    )

visualize_dataset(
    train_ds,
    value_range=(0, 255),
    rows=3,
    cols=3,
    bounding_box_format='xywh'
)

In [ ]:
total_images=1600
EPOCHS = 1
BATCH_SIZE = 8
total_steps = (total_images // BATCH_SIZE) * EPOCHS

train_ds = load(split='train', bounding_box_format='xywh')
train_ds = train_ds.ragged_batch(BATCH_SIZE)

eval_ds = load(split='test', bounding_box_format='xywh')
eval_ds = eval_ds.ragged_batch(BATCH_SIZE)

batch = next(iter(train_ds.take(1)))
keras_cv.visualization.plot_bounding_box_gallery(
    batch['images'].to_tensor(),
    y_true=batch['bounding_boxes'],
    value_range=(0, 255),
    scale=10,
    font_scale=6,
    rows=2,
    cols=4,
    class_mapping=class_mapping,
    bounding_box_format='xywh'
)

In [ ]:
augmenter = keras.Sequential(
    layers=[
        keras_cv.layers.RandomFlip(mode="horizontal", bounding_box_format="xywh"),
        keras_cv.layers.JitteredResize(
            target_size=(640, 640), scale_factor=(0.75, 1.3), bounding_box_format="xywh"
        ),
    ]
)
train_ds = train_ds.map(augmenter, num_parallel_calls=tf.data.AUTOTUNE)


In [ ]:
inference_resizing = keras_cv.layers.Resizing(
    640, 640, pad_to_aspect_ratio=True, bounding_box_format="xywh"
)
eval_ds = eval_ds.map(inference_resizing, num_parallel_calls=tf.data.AUTOTUNE)

train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
eval_ds = eval_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def line_plot(
    data,
    title=None,
    legend="auto",
    xlabel=None,
    ylabel=None,
    show=None,
    path=None,
    transparent=True,
    dpi=60,
    palette="mako_r",
):

    if show and path is not None:
        raise ValueError("Expected either `show` or `path` to be set, but not both.")
    if path is None and show is None:
        show = True
    palette = sns.color_palette("mako_r", len(data.keys()))

    sns.lineplot(data=data, palette=palette, legend=legend)
    # plt.legend(list(data.keys()))

    if xlabel:
        plt.xlabel(xlabel)
    if ylabel:
        plt.ylabel(ylabel)

    plt.suptitle(title)
    plt.show()
    plt.close()

# Clients

Global Model

In [ ]:
global_model = keras_cv.models.RetinaNet.from_preset(
    "resnet50_imagenet",
    num_classes=len(class_mapping),
    bounding_box_format="xywh"
)

In [ ]:
optimizer = optimizers.SGD(
    weight_decay=5e-4, #decay is not used anymore
    momentum=0.9,
    global_clipnorm=10.
)

global_model.compile(
    optimizer=optimizer,
    classification_loss="focal",
    box_loss="smoothl1",
)

In [ ]:
history_gm = global_model.fit(
  train_ds,
  validation_data=eval_ds,
  epochs=1,
  verbose=1
)

In [ ]:
line_plot(data=history_gm.history)

In [ ]:
global_model.save('/content/drive/MyDrive/Federated Learning/GlobalModel/global_model.keras')

In [ ]:
global_model.save_weights('/content/drive/MyDrive/Federated Learning/GlobalModel/weights_gm.weights.h5')

Client 1

In [ ]:
client1 = keras_cv.models.RetinaNet.from_preset(
    "resnet50_imagenet",
    num_classes=len(class_mapping),
    bounding_box_format="xywh"
)

In [ ]:
optimizer = optimizers.SGD(
    weight_decay=5e-4, #decay is not used anymore
    momentum=0.9,
    global_clipnorm=10.
)

client1.compile(
    optimizer=optimizer,
    classification_loss="focal",
    box_loss="smoothl1",
)

In [ ]:
history_c1 = client1.fit(
  train_ds,
  validation_data=eval_ds,
  epochs=3,
  verbose=1
)

In [ ]:
line_plot(data=history_c1.history)

In [ ]:
client1.save('/content/drive/MyDrive/Federated Learning/Client1/model_c1.keras')
client1.save_weights('/content/drive/MyDrive/Federated Learning/Client1/weights_c1.weights.h5')

Client 2

In [ ]:
client2 = keras_cv.models.RetinaNet.from_preset(
    "resnet50_imagenet",
    num_classes=len(class_mapping),
    bounding_box_format="xywh"
)

In [ ]:
optimizer = optimizers.SGD(
    weight_decay=5e-4, #decay is not used anymore
    momentum=0.9,
    global_clipnorm=10.
)

client2.compile(
    optimizer=optimizer,
    classification_loss="focal",
    box_loss="smoothl1",
)

In [ ]:
history_c2 = client2.fit(
  train_ds,
  validation_data=eval_ds,
  epochs=3,
  verbose=1
)

In [ ]:
line_plot(data=history_c2.history)

In [ ]:
client2.save('/content/drive/MyDrive/Federated Learning/Client2/model_c2.keras')
client2.save_weights('/content/drive/MyDrive/Federated Learning/Client2/weights_c2.weights.h5')

Client 3

In [ ]:
client3 = keras_cv.models.RetinaNet.from_preset(
    "resnet50_imagenet",
    num_classes=len(class_mapping),
    bounding_box_format="xywh"
)

In [ ]:
optimizer = optimizers.SGD(
    weight_decay=5e-4, #decay is not used anymore
    momentum=0.9,
    global_clipnorm=10.
)

client3.compile(
    optimizer=optimizer,
    classification_loss="focal",
    box_loss="smoothl1",
)

In [ ]:
history_c3 = client3.fit(
  train_ds,
  validation_data=eval_ds,
  epochs=3,
  verbose=1
)

In [ ]:
line_plot(data=history_c3.history)

In [ ]:
client3.save('/content/drive/MyDrive/Federated Learning/Client3/model_c3.keras')
client3.save_weights('/content/drive/MyDrive/Federated Learning/Client3/weights_c3.weights.h5')

Client 4

In [ ]:
client4 = keras_cv.models.RetinaNet.from_preset(
    "resnet50_imagenet",
    num_classes=len(class_mapping),
    bounding_box_format="xywh"
)

In [ ]:
optimizer = optimizers.SGD(
    weight_decay=5e-4, #decay is not used anymore
    momentum=0.9,
    global_clipnorm=10.
)

client4.compile(
    optimizer=optimizer,
    classification_loss="focal",
    box_loss="smoothl1",
)

In [ ]:
history_c4 = client4.fit(
  train_ds,
  validation_data=eval_ds,
  epochs=3,
  verbose=1
)

In [ ]:
line_plot(data=history_c4.history)

In [ ]:
client4.save('/content/drive/MyDrive/Federated Learning/Client4/model_c4.keras')
client4.save_weights('/content/drive/MyDrive/Federated Learning/Client4/weights_c4.weights.h5')

Weight Aggregation

In [ ]:
import h5py

# Check the contents of the weights file
with h5py.File('/content/drive/MyDrive/Fed Learn/GlobalModel/weights.h5', 'r') as f:
    print(f.keys())  # This will print the keys (datasets) present in the file

In [ ]:
# Loading the initial global model
filepath = '/content/drive/MyDrive/Federated Learning/GlobalModel/global_model.keras'
global_model_loaded = tf.keras.models.load_model(filepath)
global_model_loaded.summary()

In [ ]:
global_model_loaded.get_weights()

In [ ]:
# Loading the client models
client1 = tf.keras.models.load_model('/content/drive/MyDrive/Federated Learning/Client1/model_c1.keras')
client2 = tf.keras.models.load_model('/content/drive/MyDrive/Federated Learning/Client2/model_c2.keras')
client3 = tf.keras.models.load_model('/content/drive/MyDrive/Federated Learning/Client3/model_c3.keras')
client4 = tf.keras.models.load_model('/content/drive/MyDrive/Federated Learning/Client4/model_c4.keras')

In [ ]:
# Storing the client weights in variable
weights_c1 = client1.get_weights()
weights_c2 = client2.get_weights()
weights_c3 = client3.get_weights()
weights_c4 = client4.get_weights()

In [ ]:
# weights_gm = (weights_c1 + weights_c2 + weights_c3 + weights_c4) / 4
# global_model.set_weights(weights_gm)

In [ ]:
# Trying this method of averaging weights first
averaged_weights = []
for weight1, weight2, weight3, weight4 in zip(weights_c1, weights_c2, weights_c3, weights_c4):
    averaged_weight = (weight1 + weight2 + weight3 + weight4) / 4
    averaged_weights.append(averaged_weight)

In [ ]:
global_model_loaded.set_weights(averaged_weights)

In [ ]:
global_model_loaded.get_weights()

In [ ]:
global_model_loaded.save('/content/drive/MyDrive/Federated Learning/GlobalModel/New/global_model_new.keras')
global_model_loaded.save_weights('/content/drive/MyDrive/Federated Learning/GlobalModel/New/weights_gm_new.weights.h5')

In [ ]:
optimizer = optimizers.SGD(
    weight_decay=5e-4, #decay is not used anymore
    momentum=0.9,
    global_clipnorm=10.
)

global_model_loaded.compile(
    optimizer=optimizer,
    classification_loss="focal",
    box_loss="smoothl1",
)

In [ ]:
history_gm_loaded = global_model_loaded.fit(
  train_ds,
  validation_data=eval_ds,
  epochs=3,
  verbose=1
)

In [ ]:
line_plot(data=history_gm_loaded.history)

Homomorphic Encryption

In [ ]:
import tenseal as ts

print('Creating Private, Public Homomorphic Keys')
# controls precision of the fractional part
bits_scale = 26

# Create TenSEAL context
context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[31, bits_scale, bits_scale, bits_scale, bits_scale, bits_scale, bits_scale, 31]
)

# set the scale
context.global_scale = pow(2, bits_scale)

# galois keys are required to do ciphertext rotations
context.generate_galois_keys()
print(context)
print('Created Private, Public Homomorphic Keys')

In [ ]:
print("Seperate the Keys into public and private")

public_key = context.copy()

private_key = context.copy()

public_key.make_context_public()
print("Serialize the Public and Private Context to enable saving")
public_key_serialized = public_key.serialize(save_secret_key= False)
private_key_serialzed = private_key.serialize(save_secret_key= True)
loc = '/content/drive/MyDrive/Federated Learning/HE/'
print("Save the Public and Private Context to at the following locations")
prvt_key_loc = f'{loc}private_key_ctx.pickle'
pbl_key_loc = f'{loc}public_key_ctx.pickle'
print(f"Private Key Location: {prvt_key_loc}")
print(f"Public Key Location: {pbl_key_loc}")
with open(prvt_key_loc, 'wb') as handle:
    pickle.dump(private_key_serialzed, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(pbl_key_loc, 'wb') as handle:
    pickle.dump(public_key_serialized, handle, protocol=pickle.HIGHEST_PROTOCOL)

key_store = {'loc':loc, 'public':public_key_serialized, 'private': private_key_serialzed}

private_key = ts.context_from(key_store['private'])
public_key = ts.context_from(key_store['public'])

In [ ]:
context.generate_galois_keys()
context.global_scale = 2**40

In [ ]:
weight_vector = client_1[0]
new_weights1 = []
for i in weight_vector:
  for j in i:
    for k in j:
      for l in k:
        new_weights1.append(l)

len(new_weights1)